In [5]:
!pip install pydantic anytree networkx matplotlib inflect openapi-core jsonref prance datamodel-code-generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: inflect
    Found existing installation: inflect 7.3.1
    Uninstalling inflect-7.3.1:
      Successfully uninstalled inflect-7.3.1

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
import json

def parse_spec(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [3]:
filepath = "../openapi_specs/resolved/stripe-08-10-24.json"

spec = parse_spec(filepath)

In [50]:
# !datamodel-codegen  --input "../openapi_specs/stripe-08-10-24.yaml" --input-file-type openapi --output model.py

Modular references require an output directory, not a file


In [69]:
from pydantic import BaseModel
from collections import defaultdict


class RouteMethodDescription(BaseModel):
    method: str
    description: str


# fetch all resources
def extract_resources(openapi: dict):
    resource_to_routes = defaultdict()  # resource -> {paths} -> {methods} -> description

    for path, methods in openapi['paths'].items():
        # not a _real_ resource
        if 'get' not in methods.keys() and 'post' not in methods.keys():
            continue

        # find current resource  
        def find_last_unwrapped_element(path_list):
            for element in reversed(path_list):
                if not (element.startswith('{') and element.endswith('}')):
                    return element
            return None

        resource = find_last_unwrapped_element(path.split('/'))

        if not resource:
            continue

        if resource not in resource_to_routes:
            resource_to_routes[resource] = {}

        resource_to_routes[resource][path] = {}

        for m, v in methods.items():
            resource_to_routes[resource][path][m] = v.get('description')

    return resource_to_routes

In [116]:
extraction = extract_resources(spec)
resources = extraction.keys()

resources

dict_keys(['account', 'account_links', 'account_sessions', 'accounts', 'bank_accounts', 'capabilities', 'external_accounts', 'login_links', 'people', 'persons', 'reject', 'domains', 'application_fees', 'refunds', 'refund', 'secrets', 'delete', 'find', 'balance', 'history', 'balance_transactions', 'alerts', 'activate', 'archive', 'deactivate', 'meter_event_adjustments', 'meter_events', 'meters', 'event_summaries', 'reactivate', 'configurations', 'sessions', 'charges', 'search', 'capture', 'dispute', 'close', 'expire', 'line_items', 'orders', 'cancel', 'products', 'suppliers', 'confirmation_tokens', 'country_specs', 'coupons', 'credit_notes', 'preview', 'lines', 'void', 'customer_sessions', 'customers', 'verify', 'cards', 'cash_balance', 'cash_balance_transactions', 'discount', 'funding_instructions', 'payment_methods', 'sources', 'subscriptions', 'tax_ids', 'disputes', 'active_entitlements', 'features', 'ephemeral_keys', 'events', 'exchange_rates', 'file_links', 'files', 'disconnect', '

In [131]:
# GOAL: create resource dependency tree
# Thesis: to create a resource you need to do POST, dependents are ids and other resource names

def organize_resources(openapi):
    edges = [] # list(parent, child)

    for path, methods in openapi['paths'].items():
        # not a _real_ resource
        if 'post' not in methods.keys():
            continue
        
        path_list = path.split('/')
        
        # edges from path structure
        flag = None
        for p in path_list:
            if p in resources and flag:
                edges.append((flag, p))
            if p in resources:
                flag = p    
        
        # edges from request shape
        def extract_param_names(obj):
            param_names = []
            
            def recursive_extract(current_obj):
                if isinstance(current_obj, dict):
                    if 'properties' in current_obj:
                        param_names.extend(current_obj['properties'].keys())
                    else:
                        for value in current_obj.values():
                            recursive_extract(value)
                elif isinstance(current_obj, list):
                    for item in current_obj:
                        recursive_extract(item)
            
            recursive_extract(obj)
            return list(set(param_names))
        
        def is_in_main_list(string, main_list):
            return string in main_list
        
        def is_singular_in_main_list(string, main_list):
            return string.endswith('s') and string[:-1] in main_list
        
        def filter_strings(main_list, filter_list):
            conditions = [
                is_in_main_list,
                is_singular_in_main_list,
            ]
            
            return [
                string for string in filter_list 
                if any(condition(string, main_list) for condition in conditions)
            ]
        
        params = extract_param_names(methods['post']['requestBody'])
        for r in filter_strings(params, resources):
            edges.append((flag, r))
    
    return edges

In [132]:
edgs = organize_resources(spec)
edgs

len(edgs)

350

In [134]:
from networkx import DiGraph
import networkx as nx


def build_dependency_tree(edges: []) -> DiGraph:
    graph = nx.DiGraph()  # Directed graph allows a child to have multiple parents

    graph.add_edges_from(edges)

    return graph

In [135]:
g = build_dependency_tree(edgs)

In [136]:
g